## Imports

In [1]:
import numpy as np;
import matplotlib.pyplot as plt;
import scipy.fft as fft;
import Units
from Sequence2param import Sequence2param;

Data imports

In [2]:
filepath = "../Output/1param_acc_as2d_Dec28/";
filepath2 = "../Output/2param_acc_Option2_Dec28/";

Seq1 = Sequence2param( filepath);
Seq2 = Sequence2param( filepath2);


FileNotFoundError: ../Output/2param_acc_Option2_Dec28/Acceleration.txt not found.

### Plotting Comparison Between the Two

In [21]:
plotPaV = Seq1.BayesianUpdating(2000,5, 200)
plotPaV2 = Seq1.BayesianUpdating(2000,5, 200)

fig=plt.figure(figsize=(10,10))
gs = fig.add_gridspec(4, 2, hspace=0.2, wspace=0.1)
axs = gs.subplots(sharex='col', sharey='row')
axs[-1,-1].axis("off");
AList = Seq1.AList;
VList = Seq2.VList;

measure_points = [ 0, 20, 50 , 70];

for i in range(len(measure_points)):
    ax = axs[i, 0];
    im=ax.imshow(plotPaV[measure_points[i]], cmap = "magma" , extent =[VList[0],VList[-1],AList[0],AList[-1]], aspect=25);

    ax.set_title(str(10*measure_points[i])+" measurements")
    ax.set_xlabel("V0")
    ax.set_ylabel("a")

    ax = axs[i, 1];
    im=ax.imshow(plotPaV2[measure_points[i]], cmap = "magma" , extent =[VList[0],VList[-1],AList[0],AList[-1]], aspect=25);

    ax.set_title(str(10*measure_points[i])+" measurements")
    ax.set_xlabel("V0")
    ax.set_ylabel("a")

#fig.colorbar(im,  orientation='vertical')


for ax in fig.get_axes():
    ax.label_outer()

plt.show()

KeyboardInterrupt: 